In [1]:
import time

import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch_geometric.nn import VGAE
from torch_geometric.loader import DataLoader
from torch_geometric.utils import (degree, negative_sampling, 
                                   batched_negative_sampling)

from torch.utils.tensorboard import SummaryWriter

from gene_graph_dataset import G3MedianDataset
from phylognn_model import G3Median_GCNConv

In [2]:
gpuid = 0

train_p, test_p, val_p = 0.7, 0.2, 0.1
train_batch, test_batch, val_batch = 256, 64, 32

In [3]:
device = torch.device('cuda:' + str(gpuid) if torch.cuda.is_available() else 'cpu')

In [4]:
dataset = G3MedianDataset('dataset_g3m', 100, 100)

In [5]:
data_size = 1000 # len(dataset)
train_size, test_size, val_size = ((int)(data_size * train_p), 
                                   (int)(data_size * test_p), 
                                   (int)(data_size * val_p))

In [6]:
dataset = dataset.shuffle()
train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:(train_size + test_size)]
val_dataset = dataset[(train_size + test_size):(train_size + test_size + val_size)]

In [7]:
test_dataset = list(test_dataset)
for t in test_dataset:
    t.neg_edge_label_index = negative_sampling(t.pos_edge_label_index, 
                                        t.num_nodes,
                                        t.num_nodes**2)
train_dataset = list(train_dataset)
for t in train_dataset:
    t.neg_edge_label_index = negative_sampling(t.pos_edge_label_index, 
                                        t.num_nodes,
                                        t.num_nodes**2)
val_dataset = list(val_dataset)
for t in val_dataset:
    t.neg_edge_label_index = negative_sampling(t.pos_edge_label_index, 
                                        t.num_nodes,
                                        t.num_nodes**2)

In [8]:
train_loader = DataLoader(train_dataset, batch_size = train_batch, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = test_batch)
val_loader = DataLoader(val_dataset, batch_size= val_batch)

In [9]:
# deg = torch.zeros(5, dtype=torch.long)
# for data in train_dataset:
#     d = degree(data.edge_index[1].type(torch.int64), 
#                num_nodes=data.num_nodes, dtype=torch.long)
#     deg += torch.bincount(d, minlength=deg.numel())

In [10]:
in_channels, out_channels = dataset.num_features, 16

In [11]:
model = VGAE(G3Median_GCNConv(in_channels, out_channels)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=30,
                              min_lr=0.00001)

In [12]:
writer = SummaryWriter(log_dir='runs_g3m/g3median_lngtrain_gcn_aneg_run3')

In [13]:
from torch_geometric.data import Batch
def train(train_loader):
    model.train()
    
    total_loss = 0
    for data in train_loader:
        
        data = data.to(device)
        optimizer.zero_grad()
        
        z = model.encode(data.x, data.edge_index)
        loss = model.recon_loss(z, data.pos_edge_label_index, data.neg_edge_label_index)
        loss = loss + (1 / data.num_nodes) * model.kl_loss()
        loss.backward()
        optimizer.step()
        
        total_loss += loss
    return total_loss/len(train_loader)

In [14]:
@torch.no_grad()
def test(test_loader):
    model.eval()
    auc, ap = 0, 0
    
    for data in test_loader:
        
        data = data.to(device)
        
        z = model.encode(data.x, data.edge_index)
        # loss += model.recon_loss(z, data.pos_edge_label_index, data.neg_edge_label_index)
        tauc, tap = model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)
        
        auc += tauc
        ap += tap
        
    return auc/len(test_loader), ap/len(test_loader)

In [15]:
@torch.no_grad()
def val(val_loader):
    model.eval()
    loss = 0
    
    for data in val_loader:
        
        data = data.to(device)
        
        z = model.encode(data.x, data.edge_index)
        loss += model.recon_loss(z, data.pos_edge_label_index, data.neg_edge_label_index)
        # tauc, tap = model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)
                
    return loss/len(val_loader)

In [ ]:
for epoch in range(1, 1000 + 1):
    # print(f'{time.ctime()} - Epoch: {epoch:04d}')
    loss = train(train_loader)
    # print(f'{time.ctime()} - \t train loss: {loss:.6f}')
    tloss = val(val_loader)
    # print(f'{time.ctime()} - \t val   loss: {tloss:.6f}')
    # scheduler.step(tloss)
    
    writer.add_scalar('Loss/train', loss, epoch)
    writer.add_scalar('Loss/val', tloss, epoch)
    
    
    auc, ap = test(test_loader)
    
    writer.add_scalar('AUC/test', auc, epoch)
    writer.add_scalar('AP/test', ap, epoch)
    
    if epoch % 10 == 0:
        print(f'{time.ctime()} - Epoch: {epoch:04d}        auc: {auc:.6f}, ap: {ap:.6f}')

Sun Dec 26 23:20:09 2021 - Epoch: 0010        auc: 0.638835, ap: 0.008068
Sun Dec 26 23:21:46 2021 - Epoch: 0020        auc: 0.883538, ap: 0.048269
Sun Dec 26 23:23:23 2021 - Epoch: 0030        auc: 0.922439, ap: 0.079676
Sun Dec 26 23:24:59 2021 - Epoch: 0040        auc: 0.939210, ap: 0.100045
Sun Dec 26 23:26:35 2021 - Epoch: 0050        auc: 0.948193, ap: 0.120595
Sun Dec 26 23:28:11 2021 - Epoch: 0060        auc: 0.954467, ap: 0.136986
Sun Dec 26 23:29:50 2021 - Epoch: 0070        auc: 0.958350, ap: 0.147663
Sun Dec 26 23:31:28 2021 - Epoch: 0080        auc: 0.961869, ap: 0.157916
Sun Dec 26 23:33:06 2021 - Epoch: 0090        auc: 0.964958, ap: 0.169700
Sun Dec 26 23:34:46 2021 - Epoch: 0100        auc: 0.967392, ap: 0.179080
Sun Dec 26 23:36:22 2021 - Epoch: 0110        auc: 0.969211, ap: 0.186234
Sun Dec 26 23:37:52 2021 - Epoch: 0120        auc: 0.970613, ap: 0.191876
Sun Dec 26 23:39:20 2021 - Epoch: 0130        auc: 0.971594, ap: 0.195997
Sun Dec 26 23:40:51 2021 - Epoch: 0140

In [ ]:
writer.close()

In [ ]:
# torch.save(model.state_dict(), 'g2g_test_model_batch')